<a href="https://colab.research.google.com/github/guilp03/Music-Translator/blob/main/separador_de_musica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Fazer upload do arquivo .mp3
from google.colab import files
uploaded = files.upload()


In [7]:
import requests
import os
import time

API_KEY = "03ee442b-17c3-43aa-bd2c-09753e410fa2"

headers = {
    "Authorization": API_KEY
}

res = requests.get("https://api.music.ai/v1/upload", headers=headers)
upload_info = res.json()

upload_url = upload_info['uploadUrl']
download_url = upload_info['downloadUrl']


In [ ]:
# Caminho local do arquivo
file_path = ""

with open(file_path, "rb") as f:
    put_res = requests.put(upload_url, data=f)
    print("Status do upload:", put_res.status_code)


In [ ]:
payload = {
    "name": "Separar Stems",
    "workflow": "separador_main",
    "params": {
        "inputUrl": download_url
    }
}

job_res = requests.post(
    "https://api.music.ai/v1/job",
    headers={**headers, "Content-Type": "application/json"},
    json=payload
)

print(job_res.json())



In [ ]:
for i in range(2):
  JOB_ID = "" #copie e cole o id gerado
  res = requests.get(f"https://api.music.ai/v1/job/{JOB_ID}", headers=headers)
  time.sleep(30)
  data = res.json()
  output_dir = "/content/stems"
  os.makedirs(output_dir, exist_ok=True)
  for stem, url in data["result"].items():
        print(f"{stem}: {url}")
        local_path = f"{output_dir}/{stem}.wav"
        print(f"Baixando {stem} → {local_path}")
        with requests.get(url, stream=True) as r:
          r.raise_for_status()
          with open(local_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
        print()